# Chapter 3: Effective Theory of Deep Linear Networks at Initialization

This document will explore the claims laid out in Chapter 3 of Principles of Deep Learning Theory (it will be referred to from hereon as PDLT).
In particular, it discusses the distribution of variables contained within deep linear networks with no bias. After some mathematical analysis, it claims the following:

If the parameters in the weights &c are initialised according to Gaussian distributions, then the following should be true:
- The first layer has a distribution which is Gaussian
- Subsequent layers deviate from Gaussian, and this deviance has a scale that is related to the depth-to-width ratio.
    - If this ratio if small, its deviation from Gaussianity is linear in the ratio (if we look at the 4-point connected correlator (also known as cumulant) of the preactivation distribution)
    - If the ratio is large, its equivalent connected correlator varies exponentially from Gaussianity

It also shows that in the limit of infinite width, the neuron values become Gaussian, which implies that the contributions for a neuron from those in the previous layer cancel out.
The text claims that riding this line of complete order (where no deviance from a purely random Gaussian distribution) and chaos (where the network is so inconsistent between instantiations, and there is a lot of noise between neurons) is what defines the ability of a network to learn sufficiently complex non-linear functions.

Reading the text will hopefully help to illuminate this claim; however, this notebook will attempt to verify their claim of the above using neural networks created with Pytorch. It's worth noting that their technical definition will vary from a normal attempt of creating a deep linear network in Pytorch, due to the fact that we will need to keep track of the values of the correlators (in this case, the expectations of different monomials of the network preactivations).

## Defining the deep learning model with access to preactivations for statistical analysis

First, we need to define a neural network which is a deep linear model and which will also allow us to be able to read out the values of its preactivations at each layer. These are not usually easily accessible when creating a model, so we will have to edit the flow to make sure these can be accessed and saved.

For now, to make the maths easier, we will set the input dimension to 10, all layers to the same width of 10, and the model depth shall be 10 as well.

For the initialisation of each layer, equation (3.4) in §3 states the following choice of mean and standard deviation of the weights in the model.

More specifically, for a deep linear model with max depth $L$, given the preactivations $z_i^{(\ell)}$ for layer $\ell$, and the weights $W_{ij}$ are such that $z_i^{(\ell + 1)} = W_{ij}^{(\ell + 1)} z_j^{(\ell)}$, then the initialisation distribution for $W_{ij}^{(\ell)}$ are as follows:

$$\begin{aligned}
\mathbb{E}[W_{ij}^{(\ell)}] &= 0 \\
\mathbb{E}[W_{i_1 j_1}^{(\ell)} W_{i_2 j_2}^{(\ell)}] &= \delta_{i_1 i_2} \delta_{j_1 j_2} \frac{C_W}{n_{\ell - 1}}
\end{aligned}$$

The variance is normalised as such as this factor then cancels out when integrating over the previous layer.

So, we shall define the variables used in the model as follows:

$$\begin{aligned}
L &= 10 \\
n_{\ell} &= 10 \quad \forall \, \ell \\
C_W &= 1
\end{aligned}$$

First, Python imports need to be laid out:

In [1]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
import numpy as np

import torch
from torch import nn

In [ ]:
MODEL_DEPTH = 10
LAYER_WIDTH = 10

C_W = 1.0

class DeepLinearModel(nn.Module):
    def __init__(self) -> None:
        super().__init__()

        self.layers = [
            self.get_normal_initialised_layer()
            for _ in range(MODEL_DEPTH)
        ]

        self.z_vals = []

    def get_normal_initialised_layer() -> nn.Linear:
        layer = nn.Linear(LAYER_WIDTH, LAYER_WIDTH, bias=False)
        nn.init.normal_(layer, mean=0.0, std=C_W/LAYER_WIDTH)

        return layer